In [125]:
from numpy import array
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import copy

In [126]:
df = pd.read_csv("intrusion-detection-theory/Train_data.csv")
lbl = LabelEncoder();
df['protocol_type'] = lbl.fit_transform(df['protocol_type'])
df.shape
# df

(125973, 42)

In [127]:
Y = df['xAttack']
df.drop(['xAttack','num_outbound_cmds'],axis=1, inplace=True)
print(df.shape)
columns = df.columns
X = df.iloc[:,0:40].values
xmean = np.mean(X, axis=0)
xstd = np.std(X,axis=0)
# print(xmean)
FX = []
for index in X:
    tmp = []
    for i in range(len(index)):
        tmp.append((index[i]-xmean[i])/xstd[i])
    FX.append(tmp)
X = np.array(FX)

(125973, 40)


In [128]:
sklearn_pca = PCA(n_components=21)
Y_sklearn = sklearn_pca.fit_transform(X)
Y_sklearn

array([[-0.8495698 , -0.11068436, -0.03487308, ...,  0.56186162,
         0.6185327 , -0.44680808],
       [ 0.27017094,  1.2417322 , -0.16300179, ...,  0.03510949,
         0.72343606, -1.19488482],
       [ 4.12819764, -1.20715552,  0.05332764, ..., -0.27945382,
        -0.0457868 , -0.2531399 ],
       ...,
       [-0.15960819, -0.42488558,  0.19789581, ..., -0.65675226,
        -0.47285933, -0.7247115 ],
       [ 4.06722071, -1.27695518, -0.01965612, ...,  0.23921812,
         0.40620799,  0.07138056],
       [-1.3581793 , -0.48372761,  0.02635176, ...,  0.38645703,
         0.37672194, -0.17981096]])

In [129]:
meanvector = np.mean(X, axis=0)
m = X.shape[0]-1;
covariance = ((X - meanvector).T.dot((X - meanvector)))/m
# covariance

In [130]:
eigenvalue, eigenvector = np.linalg.eig(covariance)
eigen_val_vec = []
# eigenvalue = np.abs(eigenvalue)
for i in range(len(eigenvector)):
    t = (eigenvalue[i],eigenvector[:,i])
    eigen_val_vec.append(t)

# eigenvector
# eigenvalue

In [131]:
import operator
dictionary = dict(zip(columns, eigenvalue))
dictionary = sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True)
dictionary

[('duration', 7.700908616811359),
 ('protocol_type', 5.028775118413451),
 ('service', 2.823236887563214),
 ('flag', 2.2626637035654364),
 ('src_bytes', 1.956517812089621),
 ('dst_bytes', 1.857494291584614),
 ('land', 1.4233783473709745),
 ('wrong_fragment', 1.3022550311341843),
 ('urgent', 1.1846533326278497),
 ('dst_host_srv_count', 1.0906472645568495),
 ('dst_host_same_src_port_rate', 1.0336495732043829),
 ('dst_host_srv_diff_host_rate', 1.0070266448402927),
 ('dst_host_rerror_rate', 1.000424193473859),
 ('dst_host_srv_rerror_rate', 0.9999704791878697),
 ('dst_host_srv_serror_rate', 0.9900129483536815),
 ('dst_host_serror_rate', 0.9883910996776324),
 ('dst_host_diff_srv_rate', 0.9510913101409105),
 ('dst_host_same_srv_rate', 0.9021889753491834),
 ('dst_host_count', 0.7199273539897261),
 ('srv_diff_host_rate', 0.6733383267919277),
 ('diff_srv_rate', 0.6124309023640803),
 ('same_srv_rate', 0.6005807628444133),
 ('srv_rerror_rate', 0.4563156396411178),
 ('rerror_rate', 0.435566298296970

In [132]:
eigen_val_vec.sort()
eigen_val_vec.reverse()
# eigen_val_vec

In [133]:
tot = sum(eigenvalue)
# eigenvalue = sorted(eigenvalue, reverse=True)
pc = 0
accuracy = 0
lowerdimensionsmatrix = []
for i in eigen_val_vec:
    
    pc = pc + 1
    accuracy = accuracy + (i[0]/tot)*100
    lowerdimensionsmatrix.append(i[1])
    if accuracy>=90:
        break

pc

21

In [134]:
lowerdimensionsmatrix = np.array(lowerdimensionsmatrix)
lowerdimensionsmatrix = lowerdimensionsmatrix.T
X_low = X.dot(lowerdimensionsmatrix)
X_low

array([[-0.8495698 ,  0.11068437, -0.03487294, ...,  0.56187986,
        -0.61860963, -0.44668091],
       [ 0.27017095, -1.24173227, -0.16300186, ...,  0.03527588,
        -0.72377819, -1.19470078],
       [ 4.12819764,  1.20715551,  0.05332763, ..., -0.27942945,
         0.04573617, -0.25311015],
       ...,
       [-0.15960819,  0.4248856 ,  0.19789577, ..., -0.6568251 ,
         0.47301959, -0.72483127],
       [ 4.06722071,  1.27695517, -0.01965613, ...,  0.2392379 ,
        -0.40624852,  0.07140205],
       [-1.35817931,  0.48372761,  0.02635174, ...,  0.3864343 ,
        -0.37667482, -0.1798448 ]])

## Why PCA not applicable on the above dataset ?
* PCA is desinged for continuous variables. It tries to minimize variance (Squared deviations). The concept of squared deviations breaks down when you have binary variables.


* In our case total 44 features are there and among them more than half features are categorical (i.e 23 features are categorical).Since they are categorical they have very small variance(small range cover) and hence if we apply PCA on them then most of the categorical features will be eliminated.Because continuous features are dominated more.One more thing, Taking projection of categorical data does not make sense because individual numeric values of categorical data does not convey anything.In the above example i have applied PCA on data and most of the categorical data has been elimintaed except 1 or 2.In such case we may not able to predict effictively. So if more number of features are categorical then PCA cannot be applicable. For categorical data we can use non linear pca.